In [1]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric
from sklearn.tree import DecisionTreeRegressor
import pickle

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable
x.shape,y.shape

((506, 13), (506,))

In [10]:
def generating_samples(input_data, target_data):
    
    l=len(input_data)
    sample_index=np.random.choice(l,303,replace=False) # Sample 60 % of data form main data set(303)
    resample_index=np.random.choice(sample_index,203,replace=False) # Resample 40 % of main-data from sample data set (203)
    total_index=np.sort(np.concatenate((sample_index,resample_index))) # joining sample and resample data
    col=np.random.choice(13,(np.random.choice(np.arange(3,13))),replace=False) # column sampling
    
    total_data=input_data[total_index[:,None],col]  #total_index[:,None] None part for changing the dim
    target_data_sd=target_data[total_index]
    #print(total_data.shape,target_data_sd.shape,)
    
    return total_data,target_data_sd,sample_index,col
    

In [4]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    #sampled = (len(a)-len([str(i) for i in a])==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
    
a,b,c,d = generating_samples(x, y)
grader_samples(a,b,c,d)

True

#### Part 2

In [5]:
def sample_gen30(x,y):
    list_input_data = []
    list_output_data = []
    list_selected_row = []
    list_selected_columns = []

    for i in range(30):
        d,o,r,c=generating_samples(x, y)
        list_input_data.append(d)
        list_output_data.append(o)
        list_selected_row.append(r)
        list_selected_columns.append(c)
        
    return list_input_data,list_output_data,list_selected_row,list_selected_columns


    
list_input_data,list_output_data,list_selected_row,list_selected_columns= sample_gen30(x,y)

In [6]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

## Step 2

In [7]:
def gen_data_for_oob(list_selected_row,list_selected_columns):
    # Predicting on unseen data
    index_data_oob=[]                   #unseen data index
    for i in list_selected_row:
        s=np.unique(i)
        l1=list(np.arange(506))
        for j in s:
            l1.remove(j)
        index_data_oob.append(l1)
        
    list_data_oob=[]                    #unseen data from the index
    Y_data_oob=[]
    for i in range(30):    
        temp=x[index_data_oob[i],list_selected_columns[i].reshape(-1,1)]
        temp=temp.transpose()
        list_data_oob.append(temp)
        Y_data_oob.append(y[index_data_oob[i]])
        
    return list_data_oob,Y_data_oob

gen_data_for_oob(list_selected_row,list_selected_columns)

([array([[1.5300e+01, 2.9600e+02, 6.5750e+00, 6.5200e+01, 0.0000e+00,
          6.3200e-03],
         [1.7800e+01, 2.4200e+02, 6.4210e+00, 7.8900e+01, 0.0000e+00,
          2.7310e-02],
         [1.5200e+01, 3.1100e+02, 6.0040e+00, 8.5900e+01, 0.0000e+00,
          1.7004e-01],
         ...,
         [1.9200e+01, 3.9100e+02, 6.0190e+00, 6.5300e+01, 0.0000e+00,
          2.3912e-01],
         [2.1000e+01, 2.7300e+02, 6.1200e+00, 7.6700e+01, 0.0000e+00,
          4.5270e-02],
         [2.1000e+01, 2.7300e+02, 6.9760e+00, 9.1000e+01, 0.0000e+00,
          6.0760e-02]]),
  array([[2.9850e-02, 3.9412e+02, 5.2100e+00, ..., 6.4300e+00, 0.0000e+00,
          0.0000e+00],
         [8.8290e-02, 3.9560e+02, 1.2430e+01, ..., 6.0120e+00, 1.2500e+01,
          0.0000e+00],
         [1.7004e-01, 3.8671e+02, 1.7100e+01, ..., 6.0040e+00, 1.2500e+01,
          0.0000e+00],
         ...,
         [2.2438e-01, 3.9690e+02, 1.4330e+01, ..., 6.0270e+00, 0.0000e+00,
          0.0000e+00],
         [4.5270e-02

In [8]:
    
def model_gen_output(x,y,list_input_data,list_output_data,list_selected_row,list_selected_columns):
    from sklearn.tree import DecisionTreeRegressor
    import pickle                     # https://scikit-learn.org/stable/modules/model_persistence.html
    list_data_oob,Y_data_oob=gen_data_for_oob(list_selected_row,list_selected_columns)

    DT=DecisionTreeRegressor(max_depth=12,min_samples_split=2)#random_state=0
    DT_model=[]
    Y_output=[]
    Y_output_oob=[]
    models=[]
    for i in range(30):
        clf=DT.fit(list_input_data[i],list_output_data[i])
        models.append(pickle.dumps(clf)) 
        Y_output.append(clf.predict(x[:,list_selected_columns[i]]))
        Y_output_oob.append(clf.predict(list_data_oob[i]))
 

    Y_output=np.array(Y_output)
    Y_output_oob=np.array(Y_output_oob)
    
    return Y_output,Y_output_oob,Y_data_oob,models
    
    
Y_output,Y_output_oob,Y_data_oob,models=model_gen_output(x,y,list_input_data,list_output_data,list_selected_row,list_selected_columns)

In [9]:
def mse_oob(x,y,list_input_data,list_output_data,list_selected_row,list_selected_columns):
    Y_output,Y_output_oob,Y_data_oob,models=model_gen_output(x,y,list_input_data,list_output_data,list_selected_row,list_selected_columns)
    mse=[]
    oob=[]
    for i in range(30):
        mse.append(mean_squared_error(y, Y_output[i,:]))
        oob.append(mean_squared_error(Y_data_oob[i],Y_output_oob[i,:]))
    MSE=np.median(mse)
    OOB=np.median(oob)
    print(MSE,OOB)
    
    return MSE,OOB

mse_oob(x,y,list_input_data,list_output_data,list_selected_row,list_selected_columns)

13.350744390945597 31.722929312386896


(13.350744390945597, 31.722929312386896)

In [11]:
MSE=[]
OOB=[]
for i in range(35):
    list_input_data,list_output_data,list_selected_row,list_selected_columns= sample_gen30(x,y)
    mse,oob=mse_oob(x,y,list_input_data,list_output_data,list_selected_row,list_selected_columns)
    MSE.append(mse)
    OOB.append(oob)
    
MSE=np.array(MSE)
OOB=np.array(OOB)
    

16.44073895569664 40.6011244922762
12.61713084065083 30.667719833535262
12.584963537291536 31.140267204508472
14.789188482605805 36.69840920557252
12.49755296874743 30.691001684749107
12.642514097324531 31.239930121054908
14.636760601648511 36.3696703394741
13.906525671623562 34.12170862010821
14.910703502174407 36.8040286126368
11.318601372856849 27.78170072651019
14.983918785848221 36.68404122643992
12.258265850454684 30.00393154272113
12.956742542592773 31.658705030005482
15.370995506793614 38.17937446620695
14.833362185945333 35.95031529174186
15.485393983647015 37.76123826520853
14.773436736944008 36.07985099084214
15.478658968805178 38.03724103903424
13.631435068532546 33.58144615805026
14.523702782185726 35.473541478632924
15.443978857096388 37.72127334403004
14.365497027730001 35.145340292120096
12.788438586683032 30.765153919563016
11.968874485530119 29.139986830464785
13.015919454164408 32.170989310009816
13.484294252394555 33.38848522747698
15.620960497464818 38.835921308080

In [12]:
def ci(data):#calculating the confidence interval
    mean=data.mean()
    std=data.std()
    size=len(data)
    left_limit = np.round(mean - 2*(std/np.sqrt(size)), 3) 
    right_limit = np.round(mean + 2*(std/np.sqrt(size)), 3)
    return left_limit,right_limit

In [13]:
left,right=ci(MSE)
print("Confidence Interval Of MSE :[{} ,{}]" .format(left,right))

Confidence Interval Of MSE :[13.443 ,14.335]


In [14]:
left,right=ci(OOB)
print("Confidence Interval Of MSE :[{} ,{}]" .format(left,right))

Confidence Interval Of MSE :[33.022 ,35.249]


In [15]:
Xq=np.array([0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60])
ans=[]
for i in range(30):
    clf=pickle.loads(models[i])
    ans.append(clf.predict(Xq[list_selected_columns[i]].reshape(1,-1)))
ans=np.array(ans)
np.median(ans)

ValueError: X has 5 features, but DecisionTreeRegressor is expecting 6 features as input.